<a href="https://colab.research.google.com/github/barsha0518/barsha-_ML_1st_lab_assignment/blob/main/Labass_3_ml_barsha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy pandas scikit-learn
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score



In [7]:
!gdown 'https://drive.google.com/uc?id=1O_NwpJT-8xGfU_-3llUl2sgPu0xllOrX' -O usa_house_price.csv
data = pd.read_csv('usa_house_price.csv')

Downloading...
From: https://drive.google.com/uc?id=1O_NwpJT-8xGfU_-3llUl2sgPu0xllOrX
To: /content/usa_house_price.csv
100% 326k/326k [00:00<00:00, 125MB/s]


In [10]:
X = data.drop(columns=['Price'])
y = data['Price']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

best_beta = None
best_r2_score = -np.inf

for fold_idx, (train_index, test_index) in enumerate(kf.split(X_scaled)):

    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]

    X_train_with_bias = np.hstack([np.ones((X_train.shape[0], 1)), X_train])
    X_test_with_bias = np.hstack([np.ones((X_test.shape[0], 1)), X_test])

    beta = np.linalg.inv(X_train_with_bias.T @ X_train_with_bias) @ (X_train_with_bias.T @ y_train)

    y_pred = X_test_with_bias @ beta

    r2 = r2_score(y_test, y_pred)
    print(f"Fold {fold_idx + 1}, R2 Score: {r2}")

    if r2 > best_r2_score:
        best_r2_score = r2
        best_beta = beta

print(f"\nBest Beta (𝛽) matrix found during cross-validation:\n{best_beta}")
print(f"Best R2 Score from cross-validation: {best_r2_score}")


Fold 1, R2 Score: 0.9179971706985147
Fold 2, R2 Score: 0.9145677884802819
Fold 3, R2 Score: 0.9116116385364478
Fold 4, R2 Score: 0.9193091764960817
Fold 5, R2 Score: 0.9243869413350317

Best Beta (𝛽) matrix found during cross-validation:
[1.23161736e+06 2.30225051e+05 1.63956839e+05 1.21115120e+05
 7.83467170e+02 1.50662447e+05]
Best R2 Score from cross-validation: 0.9243869413350317


In [11]:
X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
X_train_full_with_bias = np.hstack([np.ones((X_train_full.shape[0], 1)), X_train_full])
X_test_full_with_bias = np.hstack([np.ones((X_test_full.shape[0], 1)), X_test_full])
y_pred_full = X_test_full_with_bias @ best_beta

final_r2_score = r2_score(y_test_full, y_pred_full)
print(f"Final R2 Score on 30% test data: {final_r2_score}")


Final R2 Score on 30% test data: 0.9147458156636434


QUES-2

In [13]:
X = data.drop(columns=['Price'])
y = data['Price']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y, test_size=0.44, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.68, random_state=42)

print(f"Training Set: {X_train.shape}, Validation Set: {X_val.shape}, Test Set: {X_test.shape}")

def gradient_descent(X, y, learning_rate, iterations):
    m, n = X.shape
    X = np.hstack([np.ones((m, 1)), X])
    beta = np.zeros(n + 1)

    for i in range(iterations):
        y_pred = X @ beta
        error = y_pred - y
        gradient = (X.T @ error) / m
        beta -= learning_rate * gradient

    return beta

learning_rates = [0.001, 0.01, 0.1, 1]
iterations = 1000
best_beta = None
best_r2_val = -np.inf

for lr in learning_rates:
    beta = gradient_descent(X_train, y_train, learning_rate=lr, iterations=iterations)

    X_val_with_bias = np.hstack([np.ones((X_val.shape[0], 1)), X_val])
    X_test_with_bias = np.hstack([np.ones((X_test.shape[0], 1)), X_test])

    y_val_pred = X_val_with_bias @ beta
    y_test_pred = X_test_with_bias @ beta

    r2_val = r2_score(y_val, y_val_pred)
    r2_test = r2_score(y_test, y_test_pred)

    print(f"Learning Rate: {lr}, Validation R²: {r2_val}, Test R²: {r2_test}")

    if r2_val > best_r2_val:
        best_r2_val = r2_val
        best_beta = beta
        best_lr = lr

print(f"\nBest Learning Rate: {best_lr}")
print(f"Best Beta Coefficients: {best_beta}")

Training Set: (2800, 5), Validation Set: (704, 5), Test Set: (1496, 5)
Learning Rate: 0.001, Validation R²: -1.0335339555142435, Test R²: -0.9634048206675561
Learning Rate: 0.01, Validation R²: 0.9202011879628708, Test R²: 0.9133274876797325
Learning Rate: 0.1, Validation R²: 0.9202207766800662, Test R²: 0.9133419747998835
Learning Rate: 1, Validation R²: 0.9202207766800662, Test R²: 0.9133419747998835

Best Learning Rate: 0.1
Best Beta Coefficients: [1232180.27200919  230645.88389435  165328.94019375  120045.00851908
    2945.02108903  151375.22971285]


QUES-3

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data -O car_data.csv

columns = ["symboling", "normalized_losses", "make", "fuel_type", "aspiration",
           "num_doors", "body_style", "drive_wheels", "engine_location", "wheel_base",
           "length", "width", "height", "curb_weight", "engine_type", "num_cylinders",
           "engine_size", "fuel_system", "bore", "stroke", "compression_ratio",
           "horsepower", "peak_rpm", "city_mpg", "highway_mpg", "price"]

data = pd.read_csv('car_data.csv', names=columns)
data.replace('?', np.nan, inplace=True)
data.head()


--2024-10-24 09:35:32--  https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘car_data.csv’

car_data.csv            [ <=>                ]  25.33K  --.-KB/s    in 0.06s   

2024-10-24 09:35:33 (396 KB/s) - ‘car_data.csv’ saved [25936]



,symboling,normalized_losses,make,fuel_type,aspiration,num_doors,body_style,drive_wheels,engine_location,wheel_base,...,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
0,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,NaN,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450


In [15]:
imputer_num = SimpleImputer(strategy='mean')
imputer_cat = SimpleImputer(strategy='most_frequent')

num_cols = ["normalized_losses", "bore", "stroke", "horsepower", "peak_rpm", "price"]
data[num_cols] = imputer_num.fit_transform(data[num_cols])

data.dropna(subset=['price'], inplace=True)

cat_cols = ["num_doors"]
data[cat_cols] = imputer_cat.fit_transform(data[cat_cols])

data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symboling          205 non-null    int64  
 1   normalized_losses  205 non-null    float64
 2   make               205 non-null    object 
 3   fuel_type          205 non-null    object 
 4   aspiration         205 non-null    object 
 5   num_doors          205 non-null    object 
 6   body_style         205 non-null    object 
 7   drive_wheels       205 non-null    object 
 8   engine_location    205 non-null    object 
 9   wheel_base         205 non-null    float64
 10  length             205 non-null    float64
 11  width              205 non-null    float64
 12  height             205 non-null    float64
 13  curb_weight        205 non-null    int64  
 14  engine_type        205 non-null    object 
 15  num_cylinders      205 non-null    object 
 16  engine_size        205 non

In [16]:

num_map = {'two': 2, 'four': 4, 'three': 3, 'five': 5, 'six': 6, 'eight': 8, 'twelve': 12}
data['num_doors'] = data['num_doors'].map(num_map)
data['num_cylinders'] = data['num_cylinders'].map(num_map)


data = pd.get_dummies(data, columns=['body_style', 'drive_wheels'], drop_first=True)
le = LabelEncoder()
for col in ['make', 'aspiration', 'engine_location', 'fuel_type']:
    data[col] = le.fit_transform(data[col])

data['fuel_system'] = data['fuel_system'].apply(lambda x: 1 if 'pfi' in str(x) else 0)

data['engine_type'] = data['engine_type'].apply(lambda x: 1 if 'ohc' in str(x) else 0)
data.head()


,symboling,normalized_losses,make,fuel_type,aspiration,num_doors,engine_location,wheel_base,length,width,...,peak_rpm,city_mpg,highway_mpg,price,body_style_hardtop,body_style_hatchback,body_style_sedan,body_style_wagon,drive_wheels_fwd,drive_wheels_rwd
0,3,122.0,0,1,0,2,0,88.6,168.8,64.1,...,5000.0,21,27,13495.0,False,False,False,False,False,True
1,3,122.0,0,1,0,2,0,88.6,168.8,64.1,...,5000.0,21,27,16500.0,False,False,False,False,False,True
2,1,122.0,0,1,0,2,0,94.5,171.2,65.5,...,5000.0,19,26,16500.0,False,True,False,False,False,True
3,2,164.0,1,1,0,4,0,99.8,176.6,66.2,...,5500.0,24,30,13950.0,False,False,True,False,True,False
4,2,164.0,1,1,0,4,0,99.4,176.6,66.4,...,5500.0,18,22,17450.0,False,False,True,False,False,False


In [17]:
X = data.drop(columns=['price'])
y = data['price']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)


In [18]:

regressor = LinearRegression()
regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)

r2 = r2_score(y_test, y_pred)
print(f'R² Score on Test Set: {r2}')


R² Score on Test Set: 0.804442243576259


In [19]:

pca = PCA(n_components=0.95)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

regressor_pca = LinearRegression()
regressor_pca.fit(X_train_pca, y_train)

y_pred_pca = regressor_pca.predict(X_test_pca)
r2_pca = r2_score(y_test, y_pred_pca)
print(f'R² Score on Test Set after PCA: {r2_pca}')


R² Score on Test Set after PCA: 0.7577769105521941
